# Introduction to Dolo

# Part 0: check setup

Dolo needs a working Python 3.5 environment which be downloaded from https://www.continuum.io/downloads.

Install [ruamel.yaml](https://anaconda.org/conda-forge/ruamel.yaml), [Slycot](https://anaconda.org/cwrowley/slycot) (linux, mac only) with conda:

- `conda install -c conda-forge ruamel.yaml=0.11.11`
- `conda install -c cwrowley slycot=0.2.0`

Install dolo with pip:

- `pip install dolo`

Trick: to uninstall a previous version of Dolo installed with `pip`, run *twice*: `pip uninstall dolo`.

Check that everything is in order:

In [2]:
from dolo import yaml_import, time_iteration
model = yaml_import('https://raw.githubusercontent.com/EconForge/dolo/master/examples/models/rbc.yaml')
time_iteration(model)

Check current directory:

Use `pwd`

'/home/pablo/Programming/chicago'

Change to the directory containing the notebook and the model files. Copy distant RBC model there and import it.

Use `cd`, `groot`

# Part 1: simple monetary rules and zero lower bound

This part is inspired by the [paper](https://www.ecb.europa.eu/pub/pdf/scpwps/ecbwp377.pdf?374d600f5c7511c6f962e8480cd95a37): *Optimal Monetary Policy Under Commitment with a Zero Bound on Nominal Interest Rages* from Adam and Bili

Objective : replicate the model with simple rules.

The model is defined by the following equations:

$$\pi_{t} = \beta \; \pi_{t+1} + \lambda \; y_{t} + u_{t}$$
$$y_{t} = y_{t+1} - \phi \; \left(i_{t} - \pi_{t+1}\right) + g_{t}$$
$$u_{t} = \rho_{u} \; u_{t-1} + \epsilon_{u,t}$$
$$g_{t} = \rho_{g} \; g_{t-1} + \epsilon_{g,t}$$

We will try to solve the model under the assumption that the nominal interest rate is set by a truncated Taylor Rule: 

$$i_t = \max(0, c_y \; y_t + c_{\pi} \; \pi_t )$$

We will use the `RBC` calibration from the paper:

$  \beta = 0.9913 $, $  r^{\star} =1/\beta-1 $, $  \alpha = 0.007 $, $  \lambda = 0.057 $, $  \phi = 1.00 $,
$  \rho_u = 0.36 $, $  \rho_g = 0.8 $, $  \sigma_u = 0.00171 $, $  \sigma_g = 0.00294 $

As for the Taylor Rule, as a starting point: $ c_{\pi}=1.5$, $c_y = 0.5$

## Question 1: preliminary steps

Answer the following questions:

- which model type will be used ?
- which are the states ? the controls ? the exogenous shocks ? the parameters ?

Which algorithms can be used to solve:

- the non-constrained problem
- the constrained one ?

## Question 2: write the model file

Write in that order:

- symbols
- equations
- calibration
- options:
    - distribution of shocks
    - (approximation grid)

Import and check that everything is OK.

Use: `yaml_import`.

In [1]:
from dolo import *
model = yaml_import('zlb.yaml')  # there should be no warning/error
model

Model:
------
name: "Zero Lower Bound"
type: "dtcscc"
file: "zlb.yaml

Equations:
----------

transition
 1  : 0.0000 : u = rho_u*u(-1) + epsilon_u
 2  : 0.0000 : g = rho_g*g(-1) + epsilon_g

arbitrage
 1  : 0.0000 : pi = beta*pi(1) + lam*y + u
 2  : 0.0000 : y = y(1) - phi*(i - pi(1)) + g
 3  : 0.0000 : i - (c_pi*pi + c_y*y )                 |   -rstar<=i<=inf

## Question 3: first order solution

- produce a first order approximation
- optional:
    - inspect IRFS
    - simulate and check how often constraint is hit
    - change value of `c_pi` and verify Taylor principle
    
Use: `approximate_controls`, `simulate`, `plot_irfs`, `model.calibration`  and `model.set_calibration`

## Question 4: constrained perfect solution

- reformulate the constraint as a complementarity condition
- compute a perfect foresight solution for an initial shock on $\epsilon_u$ and $\epsilon_g$
    - optional: what happens if shock happens in period $t=2$ rather than $t=0$ ?
    
Use: `deterministic_solve`, `plot_irfs`

## Question 5: global solution

- define approximation space (ex: two times ergodic variance)
- solve the model with time iteration
- (optional): remark that controls can solved in closed form as a function of expectations. Use that fact to produce a faster solution.

Use: `model.options`(inspect), `time_iteration`, `time_iteration_direct`

# Part 2: add a catastrophic shock to the RBC model

There is an implementation of the RBC model which uses a Markov Chain for exogenous shocks. It is included in the examples as `rbc_dtmscc.yaml`. 


In the orginal implementation, the law of motion for the capital is:

  $$k_t = (1-\delta)k_{t-1} + i_{t-1}$$

The productivity process depends on an AR1:

  $$z_t = \rho z_{t-1} + \epsilon_{z,t}$$

and is given by:

  $$A_t = \exp(z_t)$$
  
The goal of this exercise is to add a catastrophic event to the model. This is done by defining two additional Markov states $x_t$ and $y_t$ defined as follows:

- $x_t$ is equal to 1 in the first period the disaster hits the economy
- $y_t$ is equal to 1 as long as the economy has not recovered (probability $p$ of staying in the disaster state)
  
When there is no disaster ($x_t=y_t=0$) there is a probability $u=3\%$ that a disaster happens next period.

Given the parameters $c_k=30\%$ and  $c_z = 1 - \left( 1- c_k \right)^{1-\alpha}$ the equations are amended as follows:

$$k_t = (1-\delta)k_{t-1}*(1-c_k x_t)+ i_{t-1}$$

$$A_t = \exp(z_t)*(1-c_z y_t)$$

Save `rbc_dtmscc.yaml` as `rbc_disaster.yaml` and add the above modifications. Solve and simulate the new version.

Use `time_iteration`, `plot_decision_rule`, `simulate`
